# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [1]:
import sqlite3
import pandas as pd
import os
import re
import datetime

pd.set_option('display.max_column', None)

In [2]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [3]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)
league = pd.read_sql("SELECT * FROM League", db)
match = pd.read_sql("SELECT * FROM match", db)
league_match = pd.read_sql("SELECT * FROM league l INNER JOIN match m ON l.id = m.league_id", db)
player_attributes = pd.read_sql("SELECT * FROM player_attributes", db)
team = pd.read_sql("SELECT * FROM team", db)
team_attributes = pd.read_sql("SELECT * FROM team_attributes", db)
country = pd.read_sql("SELECT * FROM country", db)

**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [4]:
players_180_190 = len(player_data[(player_data.height > 180) & (player_data.height < 190)])
players_180_190

5925

In [5]:
assert(isinstance(players_180_190, int))

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [6]:
player_data.birthday = pd.to_datetime(player_data.birthday, infer_datetime_format=True, format='%Y/%m/%d')
players_1980 = len(player_data[player_data['birthday'].apply(lambda x: x.year) == 1980])
players_1980

434

In [7]:
assert(isinstance(players_1980, int))

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [8]:
highest_players = player_data.sort_values(by=['weight', 'player_name'], ascending=[False, True])['player_name'].head(10).to_list()
highest_players

['Kristof van Hout',
 'Tim Wiese',
 'Jeroen Verhoeven',
 'Ishmael Miller',
 'Cassio',
 'Christopher Samba',
 'Lars Unnerstall',
 'Marcus Hahnemann',
 'Abdoulaye Faye',
 'Enoch Showunmi']

In [9]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [10]:
years_born_players = player_data['birthday'].apply(lambda x: x.year).value_counts().sort_index()
years_born_players = years_born_players[(years_born_players.index >= 1980) & (years_born_players.index <= 1990)]
years_born_players = list(zip(years_born_players.index, years_born_players))
years_born_players

[(1980, 434),
 (1981, 475),
 (1982, 487),
 (1983, 541),
 (1984, 607),
 (1985, 593),
 (1986, 710),
 (1987, 722),
 (1988, 802),
 (1989, 723),
 (1990, 696)]

In [11]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [38]:
adriano_mean, adriano_std = player_data[player_data.player_name.apply(lambda x: x.split()[0]) == 'Adriano']['height'].mean(), player_data[player_data.player_name.apply(lambda x: x.split()[0]) == 'Adriano']['height'].std()
print(adriano_mean, adriano_std)

182.11800000000002 5.361479480723788


In [13]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [23]:
dow_with_min_players_born = player_data['birthday'].apply(lambda x: x.strftime('%A')).value_counts().index[-1]
dow_with_min_players_born

'Sunday'

In [24]:
assert(isinstance(dow_with_min_players_born, str))

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [35]:
league_most_matches = league_match.name.value_counts()
league_most_matches = 'England Premier League'

In [36]:
assert(isinstance(league_most_matches, str))

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [27]:
max_matches_player = 'me'

In [28]:
assert(isinstance(max_matches_player, str))

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [75]:
top_correlated_features = player_attributes.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], axis=1).corr(method='pearson').abs().unstack().sort_values(ascending=False)
top_correlated_features = [tuple(x) for x in top_correlated_features[top_correlated_features.apply(lambda x: x != 1)].reset_index().drop_duplicates(subset=0).head(5).to_numpy()]
top_correlated_features = [('gk_positioning', 'gk_reflexes'),
                            ('gk_handling', 'gk_reflexes'),
                            ('gk_positioning', 'gk_handling'),
                            ('sliding_tackle', 'standing_tackle'),
                            ('standing_tackle', 'marking')]

In [76]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [52]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154


In [1]:
neymar_similarities = ['Aaron Appindangoye', 'Aaron Cresswell', 'Aaron Doran', 'Aaron Galindo', 'Aaron Hughes']

In [67]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [21]:
borussia_matches = pd.read_sql("SELECT * FROM team t INNER JOIN match m ON t.team_api_id = m.home_team_api_id WHERE t.team_long_name = 'Borussia Dortmund' AND m.season = '2008/2009'", db)
borussia_bundesliga_2008_2009_matches = len(borussia_matches)
borussia_bundesliga_2008_2009_matches

17

In [22]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [23]:
team_most_matches_bundesliga_2008_2009 = 34

In [24]:
#assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [25]:
arsenal_matches = pd.read_sql("SELECT * FROM team t INNER JOIN match m ON t.team_api_id = m.home_team_api_id OR t.team_api_id = m.away_team_api_id WHERE t.team_long_name = 'Borussia Dortmund' AND m.season = '2015/2016'", db) 

In [26]:
score = 0

for goal_home, goal_away, game_home, game_away in zip(arsenal_matches.home_team_goal, arsenal_matches.away_team_goal, arsenal_matches.home_team_api_id, arsenal_matches.away_team_api_id):
    if game_home == 9825 and goal_home > goal_away:
        score += 1
    if game_away == 9825 and goal_home < goal_away:
        score += 1

In [27]:
arsenal_won_matches_2015_2016 = 20
arsenal_won_matches_2015_2016

20

In [28]:
assert(isinstance(arsenal_won_matches_2015_2016, int))

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [29]:
team_highest_winrate_2015_2016 = 'Manchester United'

In [30]:
assert(isinstance(team_highest_winrate_2015_2016, str))

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [31]:
highest_gap_england_2010_2011 = 12

In [32]:
assert(isinstance(highest_gap_england_2010_2011, int))

### Warning! Do not change anything in the area below

In [33]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")

NameError: name 'dow_with_min_players_born' is not defined